#  **Análisis y Predicción de Datos Meteorológicos con Métodos de Interpolación**
### Alejandro Chávez - 32.278.392
### Cesar Dominguez - 30.640.838
### José Santana - 31.608.829

En el presente proyecto estaremos analizando con distintos métodos de Interpolación, las temperaturas de una ciudad, con el fin de modelar e intentar predecir estos datos meteorológicos.

Los metodos de interpolación que emplearemos son:
* Interpolación de Taylor
* Interpolación de Lagrange
* Interpolación de Hermite
* Interpolación Polinómica a Trozos
---
## **Generación de datos**

Para la generación de datos utilizaremos como referencia la ciudad de Valencia, Carabobo (Ubicada en Venezuela) generando datos aleatorios dentro de un rango comprendido por el valor de la temperatura mínima (20°c) y el valor de la temperatura máxima (30°c) de la última semana en la ciudad.

In [ ]:
lista = []
for i in 1:7
    push!(lista,rand(20:30))
end
println(lista)

listafija = [29, 21, 28, 24, 20, 21, 30]